<a href="https://colab.research.google.com/github/soodeh-nilforoushan/NLP-Transformers/blob/main/Fine_Tuning_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np

In [3]:
#glue datset and subtask sst2
raw_datasets=load_dataset("glue","sst2")

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [5]:
raw_datasets['train']

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [6]:
#to see what attribute and objects have
dir(raw_datasets['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_tables_from_cache_file',
 '_generate_tables_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_with_indices_mappin

In [7]:
type(raw_datasets['train'])

datasets.arrow_dataset.Dataset

In [8]:
raw_datasets['train'].data

MemoryMappedTable
sentence: string
label: int64
idx: int32
----
sentence: [["hide new secretions from the parental units ","contains no wit , only labored gags ","that loves its characters and communicates something rather beautiful about human nature ","remains utterly satisfied to remain the same throughout ","on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ",...,"you wish you were at home watching that movie instead of in the theater watching this one ","'s no point in extracting the bare bones of byatt 's plot for purposes of bland hollywood romance ","underdeveloped ","the jokes are flat ","a heartening tale of small victories "],["suspense , intriguing characters and bizarre bank robberies , ","a gritty police thriller with all the dysfunctional family dynamics one could wish for ","with a wonderful ensemble cast of characters that bring the routine day to day struggles of the working class to life ","nonetheless appreciates the art and reveals a music sc

In [9]:
#index dataset to see if it list or array
raw_datasets["train"][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [10]:
#index dataset to see range of indx
raw_datasets["train"][50000:50003]

{'sentence': ['glow ',
  'a classical dramatic animated feature ',
  'best espionage picture '],
 'label': [1, 1, 1],
 'idx': [50000, 50001, 50002]}

In [11]:
raw_datasets['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [12]:
from transformers import AutoTokenizer

In [13]:
# checkpoint="bert-base-uncased"
checkpoint="distilbert-base-uncased" #it trains faster
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [14]:
#test tokizer on subset of sentences
tokenized_sentences=tokenizer(raw_datasets["train"][0:3]["sentence"])
from pprint import pprint
pprint(tokenized_sentences)
# pprint(tokenizer(raw_datasets["sentence"]))

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
               [101,
                3397,
                2053,
                15966,
                1010,
                2069,
                4450,
                2098,
                18201,
                2015,
                102],
               [101,
                2008,
                7459,
                2049,
                3494,
                1998,
                10639,
                2015,
                2242,
                2738,
                3376,
                2055,
                2529,
                3267,
                102]]}


In [15]:
def tokenize_fn(batch):
   return tokenizer(batch['sentence'], truncation=True)

In [16]:
tokenized_datasets=raw_datasets.map(tokenize_fn, batched=True)
#input_ids: integer represenation of each toke,
#attention_mask: tells our model which tokens should enter into our attention computation token
#token_type_ids: it wont show up for every model we use

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [17]:
#thi is an example of map function
import pandas as pd
import numpy as np
technologies= {
    'Fee' :[22000,25000,23000,np.NaN,26000],
    'Duration':['30days','50days','30days','35days','40days']
          }
df = pd.DataFrame(technologies)
print(df)

# Using custom function
def fun1(x):
    return x/100
df['Fee'] = df['Fee'].map(lambda x:fun1(x))
print(df)


       Fee Duration
0  22000.0   30days
1  25000.0   50days
2  23000.0   30days
3      NaN   35days
4  26000.0   40days
     Fee Duration
0  220.0   30days
1  250.0   50days
2  230.0   30days
3    NaN   35days
4  260.0   40days


In [18]:
from transformers import TrainingArguments


In [19]:
!pip install transformers[torch]

In [20]:
!pip install datasets transformers==4.28.0

In [21]:
training_args= TrainingArguments(
    'my_trainer',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=1,
)

In [22]:
from transformers import AutoModelForSequenceClassification

In [23]:
#specify check point and number of labels
model=AutoModelForSequenceClassification.from_pretrained(
    checkpoint,num_labels=2
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.

In [24]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [25]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [26]:
!pip install torchinfo

In [27]:
from torchinfo import summary

summary(model)
#

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

In [28]:
#we are gonna do sanity check here, when we fine tune transfer model we end up all the weights in NN, not just the new untrained weights
#in order to verify that is true, we are going to save all the model parameters before we begin the training process
#these are stored inside a list called params before
params_before=[]
for num,p in model.named_parameters():
  params_before.append(p.detach().cpu().numpy())

In [29]:
from transformers import Trainer

In [30]:
from datasets import load_metric

In [31]:
metric = load_metric("glue","sst2")

<ipython-input-31-159f096838fb>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue","sst2")


In [32]:
#just test metric
metric.compute(predictions=[1,0,1], references=[1,0,0])

{'accuracy': 0.6666666666666666}

In [42]:
from transformers.pipelines.object_detection import Predictions
#if we do not wanna use thr above functionfor computing metric we can write as below
def compute_metrics(logits_and_labels):
  #metric=load_metric("glue","sst2")
  logits,labels=logits_and_labels
  #we call argmax to get prediction
  predictions =np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [43]:
#create trainer objects
trainer=Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [44]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.180100,0.388056,0.897936


TrainOutput(global_step=8419, training_loss=0.1504045888128438, metrics={'train_runtime': 472.2423, 'train_samples_per_second': 142.615, 'train_steps_per_second': 17.828, 'total_flos': 518400815624736.0, 'train_loss': 0.1504045888128438, 'epoch': 1.0})

In [50]:
my_saved_model=trainer.save_model('my_saved_model')

In [51]:
!ls

my_saved_model	my_trainer  sample_data


In [52]:
#it also shows the tokenized objects as well
!ls my_saved_model

config.json	   special_tokens_map.json  tokenizer.json     vocab.txt
pytorch_model.bin  tokenizer_config.json    training_args.bin


In [53]:

from transformers import pipeline

In [54]:
#shows how to use trained model
newmodel=pipeline('text-classification', model=my_saved_model, device=0)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [55]:
newmodel('this movie is great')

[{'label': 'POSITIVE', 'score': 0.9998806715011597}]

In [56]:
newmodel("this movie is sucks")

[{'label': 'NEGATIVE', 'score': 0.9995971322059631}]

In [57]:
!cat my_saved_model/config.json
#it does not have any information about label names

{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.28.0",
  "vocab_size": 30522
}


In [58]:
import json

In [59]:
#edit config file, we start by loading in the file and using the JSON library to convert it into dictionary
config_path="my_saved_model/config.json"
with open(config_path) as f:
  j=json.load(f)

#add new key "id2label"
j["id2label"] ={0:"negative", 1:"positive"}

#save json to the same file path
with open(config_path,"w") as f:
  json.dump(j,f,indent=2)

In [60]:
!cat my_saved_model/config.json

{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.28.0",
  "vocab_size": 30522,
  "id2label": {
    "0": "negative",
    "1": "positive"
  }
}

In [61]:
newmodel=pipeline('text-classification', model=my_saved_model, device=0)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [62]:
newmodel("this movie is great!")

[{'label': 'POSITIVE', 'score': 0.9998760223388672}]

In [63]:
#check all the weight is NN were changed during training
params_after=[]
for num,p in model.named_parameters():
  params_after.append(p.detach().cpu().numpy())

In [65]:
#compute the sum of the absolute difference of each parameters
for p1,p2 in zip(params_before, params_after):
  print(np.sum(np.abs(p1-p2)))

21854.838
131.31412
2.2620854
1.6408607
1813.3048
2.1845222
1805.0408
0.0032404196
1640.5217
1.4742444
1570.8892
1.07329
2.3171227
1.0820019
6934.5796
7.4010606
6491.186
1.0181324
2.1798022
1.019667
1798.0886
2.2304454
1811.0061
0.003510075
1608.0326
1.156972
1547.0006
0.9802971
2.1621585
0.97790647
6864.796
7.211276
6326.52
0.8843732
2.0734062
1.0518302
1754.8737
1.9818411
1758.6335
0.0029087586
1566.9076
1.0805643
1544.0813
0.9463848
2.1709337
1.0189571
6913.629
7.3475804
6222.9604
0.9382678
2.0457788
0.93123674
1758.4867
1.935921
1772.6753
0.003832383
1588.6118
0.9837852
1526.8043
0.96798027
2.0807667
1.0103439
6841.212
7.670922
5928.959
0.960734
1.8987682
0.79818404
1674.2109
1.9455042
1678.5692
0.0022501245
1409.6274
0.91588134
1421.0663
1.1173422
2.012244
1.1618617
6436.0327
7.809883
5185.973
1.1091294
1.7964667
1.3061723
1618.1672
1.8943726
1611.3999
0.0013819063
1326.1918
1.6345828
1342.8752
1.6585181
1.8635027
1.6743934
5308.07
6.9611473
4858.4043
1.4652953
2.5825486
0.9077427

As we can see all of these sums are non-zero, it means all of the model weights were updated during the training process